In [12]:
import sys 
sys.path.append("../")

import numpy as np 
from pprint import pprint
from typing import TypeVar, Iterable, Callable, Sequence
import matplotlib.pyplot as plt 
from rl.markov_process import NonTerminal, TransitionStep
import itertools
from rl.markov_decision_process import FiniteMarkovDecisionProcess
from rl.distribution import Choose
from rl.function_approx import LinearFunctionApprox, Weights
import rl.policy as policy

### Question 1: implementing LSTD algorithm from scratch

In [22]:
S = TypeVar("S")
A = TypeVar("A")

def LSTD(traces : Iterable[TransitionStep[S]], feature_functions : Sequence[Callable[[NonTerminal[S]], float]],
         gamma : float, epsilon : float) -> LinearFunctionApprox[NonTerminal[S]]:
    num_features : int = len(feature_functions)
    A_inv : np.array = np.eye(num_features)
    b_vec : np.array = np.zeros(num_features)

    for step in traces:
        state, next_state, reward = step.state, step.next_state, step.reward
        u = np.array([f(state) for f in feature_functions])
        v = u

        if isinstance(next_state, NonTerminal):
            v -= gamma * np.array([f(next_state) for f in feature_functions])

        # updating the inverse of A using Sherman-Morrison-Woodbury formula
        A_inv -= (A_inv @ np.outer(u, v) @ A_inv) / (1 + v @ A_inv @ u)
        b_vec += reward * u
    
    # find the optimal weights
    w_star : np.array = A_inv @ b_vec
    return LinearFunctionApprox.create(feature_functions=feature_functions, weights=Weights.create(w_star))

#### Testing the LSTD algorithm on an already-solved MRP

In [23]:
from rl.chapter2.simple_inventory_mrp import *

user_capacity = 2
user_poisson_lambda = 1.0
user_holding_cost = 1.0
user_stockout_cost = 10.0
user_gamma = 0.9

si_mrp = SimpleInventoryMRPFinite(capacity=user_capacity,
                                  poisson_lambda=user_poisson_lambda,
                                  holding_cost=user_holding_cost,
                                  stockout_cost=user_stockout_cost)

print("Exact Value Function:")
print("--" * 20)
si_mrp.display_value_function(gamma=user_gamma)
print()

Exact Value Function:
----------------------------------------
{NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -30.345,
 NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -27.932,
 NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -35.511,
 NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -28.932,
 NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -29.345,
 NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -28.345}



In [24]:
num_episodes = 1000
num_steps = 100
epsilon = 0.01

ffs = [(lambda _ : 1.0), (lambda s : s.state.on_hand), (lambda s : s.state.on_order)]
traces = itertools.chain.from_iterable(si_mrp.reward_traces(Choose(si_mrp.non_terminal_states)))

lstd_qf = LSTD(traces=itertools.islice(traces, num_episodes * num_steps), feature_functions=ffs, gamma=user_gamma, epsilon=epsilon)

pprint({s : round(lstd_qf(s), 3) for s in si_mrp.non_terminal_states})

{NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -31.081,
 NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -28.861,
 NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -31.003,
 NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -31.042,
 NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -28.9,
 NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -26.718}


### Question 3: implementing LSPI for American options-pricing